In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import re
import matplotlib.pyplot as plt

Running the different scripts we have in the folder, we obtained this folder data/info/bachelor with all the .xls files from 2007 and on, over which we will want to run this analysis script.
The first thing we do is to load the data, after some preliminary checks: is the file structure coherent from one file to another? We check this with a print of the column names each file has. The result is encouraging since the files share the same structure. 

Having the data, we knew we only needed to keep a few columns: the gender, the identity (sciper to ease things up and avoid people with the same name being an issue) and the presence.

The first two seem to be logical indices, and the gender is more macroscopic than the ID. Thus in the dataframe extracted from a single .xls, we ditch everything else than the 3 previously mentionned, and hierarchically reindex the rows : gender -> index.

We are left with a single column: the Status. When we tried to merge the DFs together, we realized it would be convenient to have a column per semester and year. This seems logical to visualize and individual's progression within EPFL. The semester and year can be extracted from the file's name, and as a value we can keep the presence. The latter can be either: Present, Away (or NaN). By monitoring the succession of presences, we can determine the time it took to a given student to get his bachelors.


&& TO DO 

The last step we will have to do is to remove all id's that have not finished EPFL, and do averages for the indices Female and Male separately or together, or whatever we are supposed to do.

An issue that occured to us afterwards is: what if someone is registered in BA1 in 2007 but has done it's first BA1 in 2006 ? 




In [24]:
#In this block we get the data and wrangle it

folder_path = "data/info/bachelor"
students = pd.DataFrame([])
for file_path in glob.iglob(folder_path+'/*.XLS'):
        tmp_data = pd.read_html(file_path, index_col=[0, 10], header=2)[0] #excel format is corrupted, but reading them as html works
        #We extract the Year and semester from the file's name
        num_sem = float(re.sub("[^0-9]", "", str(file_path).strip())[-2])
        if num_sem % 2 == 0:
            Semester = "Summer"
            Year = re.sub("[^0-9]", "", str(file_path).strip())[4:8]
        else:
            Year = re.sub("[^0-9]", "", str(file_path).strip())[:4]
            Semester = "Winter"
        #We sort by index, the most macroscopic index being the gender
        tmp_data= tmp_data[["Statut"]].sort_index()
        #We rename the columns so as to have the season and year instead of something called "Status"
        tmp_data.columns=[[Year], [Semester]]
        students = pd.concat([students, tmp_data])
                
students.sort_index(axis=1)
#students.describe()


2007   2008          2009          2010          2011  \
                     Winter Summer Winter Summer Winter Summer Winter Summer   
Civilité No Sciper                                                             
Madame   209429         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         213781         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         216801         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         217400         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         217559         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         218428         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         219665         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         225388         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         225642         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         226834         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         228116         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         228149         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
Monsieur 202704         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         202973         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         203066         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         203117         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         203583         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         204324         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         204869         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         204988         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         205364         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         205565         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         205727         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         208024         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         208416         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         212464         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         212480         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         212608         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         212686         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         212726         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
         166746     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         166748     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         166751     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         166889     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167007     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167045     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167313     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167386     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167439     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167447     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         167530     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         169655     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         170219     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         170220     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         170241     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         170242     Présent    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         170263     Pré